# 🎮 Video Game Sales Analytics (Clustering + PCA)

**Objective:** Analyze global video game sales data to identify **regional demand patterns** using clustering and visualization.

**How to run:**
1. Place `vgsales.csv` in `data/` (or update `CSV_PATH`).
2. Run cells top-to-bottom.

> Note: This notebook is a cleaned, portfolio-ready version of an MBA data analytics project.



## 1) Setup

- If running in **Google Colab**, upload `vgsales.csv` to the session or mount Drive and set the path accordingly.


In [ ]:

# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

print("Libraries imported.")



## 2) Load Dataset
- Replace the path if needed. If using Colab, either upload the CSV and use `/content/vgsales.csv` or point to your Drive path.


In [ ]:

CSV_PATH = 'data/vgsales.csv'  # update if your file is elsewhere  # change if needed

# Load
df_raw = pd.read_csv(CSV_PATH)
print(df_raw.shape)
df_raw.head(10)



## 3) Data Overview & Quality Checks
- Show columns, dtypes, missing values, and duplicates.
- Decide which columns to keep (we're **excluding `Year`**).


In [ ]:

print("Columns:", list(df_raw.columns))
print("\nDtypes:\n", df_raw.dtypes)
print("\nMissing values per column:\n", df_raw.isna().sum())
print("\nDuplicate rows:", df_raw.duplicated().sum())



## 4) Data Cleaning (without `Year`)
**Cleaning decisions:**
- Drop low-value/identifier columns: `Rank`
- **Exclude `Year`** per your project decision
- Trim whitespace from text columns
- Drop exact duplicates on key identifiers
- Handle missing values (drop where essential features are missing)
- Ensure sales columns are numeric


In [ ]:

df = df_raw.copy()

# 1) Drop columns (ignore if not present)
df.drop(columns=['Rank', 'Year'], errors='ignore', inplace=True)

# 2) Strip whitespace in column names & string cells
df.columns = df.columns.str.strip()
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype(str).str.strip()

# 3) Coerce sales columns to numeric (safe-guard)
sales_cols = [c for c in df.columns if c.endswith('_Sales') or c.lower() == 'global_sales']
for c in sales_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# 4) Drop duplicates based on key identifiers (tune if needed)
key_cols = [c for c in ['Name','Platform','Genre','Publisher'] if c in df.columns]
if key_cols:
    before = df.shape[0]
    df = df.drop_duplicates(subset=key_cols)
    print(f"Removed {before - df.shape[0]} duplicate rows based on {key_cols}.")

# 5) Handle missing values: drop rows missing essential fields
essential = [c for c in ['Name','Platform','Genre','Publisher','Global_Sales'] if c in df.columns]
df = df.dropna(subset=essential)

# 6) Confirm Global_Sales exists; if not, create from regionals
if 'Global_Sales' not in df.columns:
    regionals = [c for c in ['NA_Sales','EU_Sales','JP_Sales','Other_Sales'] if c in df.columns]
    if regionals:
        df['Global_Sales'] = df[regionals].sum(axis=1)

print("Shape after cleaning:", df.shape)
df.head(10)



## 5) Exploratory Data Analysis (EDA)
Focus on **genres**, **platforms**, and **titles** without using `Year`.


In [ ]:

# --- EDA: Global Sales by Genre, Platform, and Top 10 Games ---

def barplot_series(s, title, xlabel=None, ylabel=None, top=None, rotation=45):
    if top:
        s = s.head(top)
    plt.figure()
    s.plot(kind='bar')
    plt.xticks(rotation=rotation, ha='right')
    if title: plt.title(title)
    if xlabel: plt.xlabel(xlabel)
    if ylabel: plt.ylabel(ylabel)
    plt.tight_layout()
    plt.show()

# Global sales by Genre
if 'Genre' in df.columns and 'Global_Sales' in df.columns:
    genre_sales = df.groupby('Genre')['Global_Sales'].sum().sort_values(ascending=False)
    barplot_series(genre_sales, "Global Sales by Genre", ylabel="Sales (Millions)")

# Global sales by Platform (top 12)
if 'Platform' in df.columns and 'Global_Sales' in df.columns:
    platform_sales = df.groupby('Platform')['Global_Sales'].sum().sort_values(ascending=False)
    barplot_series(platform_sales, "Global Sales by Platform (Top 12)", ylabel="Sales (Millions)", top=12)

# Top 10 best-selling games overall (ranked 1–10)
cols = [c for c in ['Name','Platform','Genre','Publisher','Global_Sales'] if c in df.columns]
top10 = (
    df.sort_values('Global_Sales', ascending=False)
      .head(10)[cols]
      .reset_index(drop=True)
)
top10.index = top10.index + 1   # shifts index from 0–9 to 1–10
top10.index.name = "Rank"
display(top10)



## 6) K-Means Clustering on Regional Sales

**Goal:** Identify clusters of games based on their regional sales patterns  
(`NA_Sales`, `EU_Sales`, `JP_Sales`, `Other_Sales`).

We will:

1. Use the **Elbow Method** to choose the number of clusters (k).  
2. Fit a K-Means model on regional sales.  
3. Summarize each cluster with **average sales per region**.  
4. Visualize clusters using:
   - A **heatmap** of average sales by cluster and region  
   - A **radar (spider) chart** of regional profiles  
   - A **bar chart** comparing regions across clusters



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Columns we use for clustering
regional_cols = ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]

# Feature matrix (fill missing with 0 just in case)
X = df[regional_cols].fillna(0.0).values

# Scale features for K-Means
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# -----------------------
# Elbow Method to choose k
# -----------------------
inertias = []
K = range(2, 10)

for k in K:
    km_tmp = KMeans(n_clusters=k, random_state=42, n_init=10)
    km_tmp.fit(X_scaled)
    inertias.append(km_tmp.inertia_)

plt.figure(figsize=(6,4))
plt.plot(list(K), inertias, marker="o")
plt.title("Elbow Method: Choose k")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Inertia (Within-Cluster SSE)")
plt.grid(True)
plt.tight_layout()
plt.show()

# -----------------------
# Fit final K-Means (k=3 from elbow plot)
# -----------------------
k = 3
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
df["Cluster"] = kmeans.fit_predict(X_scaled)

# Cluster sizes
print("=== Cluster Sizes ===")
print(df["Cluster"].value_counts().sort_index(), "\n")

# Cluster profile: mean sales per region
cluster_means = (
    df.groupby("Cluster")[regional_cols]
      .mean()
      .round(2)
)

cluster_means["Total_Titles"] = df["Cluster"].value_counts().sort_index()
print("=== Cluster Profile (Average Sales per Region, in millions) ===")
display(cluster_means)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# We will reuse cluster_means and regional_cols from the previous cell

# -------------------------------------------------------
# 1) HEATMAP: Average Regional Sales by Cluster
# -------------------------------------------------------
plt.figure(figsize=(7,4))
plt.imshow(cluster_means[regional_cols], aspect="auto")
plt.colorbar(label="Average Sales (millions)")
plt.xticks(
    ticks=np.arange(len(regional_cols)),
    labels=regional_cols
)
plt.yticks(
    ticks=np.arange(len(cluster_means)),
    labels=[f"Cluster {i}" for i in cluster_means.index]
)
plt.title("Heatmap: Average Regional Sales by Cluster")
plt.tight_layout()
plt.show()

# -------------------------------------------------------
# 2) RADAR / SPIDER CHART: Regional Profile by Cluster
# -------------------------------------------------------
regions = regional_cols
angles = np.linspace(0, 2*np.pi, len(regions), endpoint=False).tolist()
angles += angles[:1]  # close the circle

plt.figure(figsize=(7,7))

for i, (idx, row) in enumerate(cluster_means[regional_cols].iterrows()):
    values = row.tolist()
    values += values[:1]
    plt.polar(angles, values, linewidth=2, label=f"Cluster {idx}")

# Improve spacing to avoid overlap
plt.xticks(angles[:-1], regions, fontsize=12)

# Move title upward
plt.title(
    "Radar Chart: Regional Sales Profile by Cluster",
    y=1.12,                 # moves title higher
    fontsize=14
)

# Optional tweak: rotate the top label slightly (EU_Sales)
labels = plt.gca().get_xticklabels()
labels[1].set_rotation(10)  # adjust angle if needed

plt.legend(loc="upper right", bbox_to_anchor=(1.3, 1.1))
plt.tight_layout()
plt.show()


# -------------------------------------------------------
# 3) BAR CHART: Average Regional Sales per Cluster
# -------------------------------------------------------
plt.figure(figsize=(8,5))
x = np.arange(len(cluster_means.index))  # positions for clusters
width = 0.2

for i, region in enumerate(regional_cols):
    plt.bar(
        x + i*width,
        cluster_means[region].values,
        width=width,
        label=region
    )

plt.xticks(x + width * (len(regional_cols)-1) / 2, [f"Cluster {i}" for i in cluster_means.index])
plt.xlabel("Cluster")
plt.ylabel("Average Sales (millions)")
plt.title("Bar Chart: Average Regional Sales per Cluster")
plt.legend()
plt.tight_layout()
plt.show()


## 7) Best Sellers by Console & Genre (Tables)



In [ ]:

# --- Best Sellers by Console & Genre (Full Tables) ---

def top_n_by_group(df, group_col, value_col='Global_Sales', n=5, extra_cols=None):
    """Return top-n rows within each group (e.g., top 5 games per genre/platform)."""
    if extra_cols is None:
        extra_cols = []
    frames = []
    for g, sub in df.groupby(group_col):
        cols = [c for c in ['Name', group_col, value_col] + extra_cols if c in sub.columns]
        top_n = sub.sort_values(value_col, ascending=False).head(n)[cols]
        top_n[group_col] = g
        frames.append(top_n)
    return pd.concat(frames, ignore_index=True)

# Top 5 games per Platform (shows ALL platforms)
if 'Platform' in df.columns and 'Global_Sales' in df.columns:
    top5_platform = top_n_by_group(df, 'Platform', 'Global_Sales', n=5, extra_cols=['Genre','Publisher'])
    print(f"Total Platforms: {df['Platform'].nunique()}")
    display(top5_platform.sort_values(['Platform', 'Global_Sales'], ascending=[True, False]))

# Top 5 games per Genre (shows ALL genres)
if 'Genre' in df.columns and 'Global_Sales' in df.columns:
    top5_genre = top_n_by_group(df, 'Genre', 'Global_Sales', n=5, extra_cols=['Platform','Publisher'])
    top5_genre[top5_genre['Genre'].isin(['Action', 'Sports', 'Shooter' 'Role-Playing', 'Platform'])]
    print(f"Total Genres: {df['Genre'].nunique()}")
    display(top5_genre.sort_values(['Genre', 'Global_Sales'], ascending=[True, False]))




## 8) Insights & Recommendations
- **Genre dominance:** Action, Sports, and Shooter genres dominate.
- **Platform dominance:** Nintendo and Playstation consoldes lead globally.
- **Regional patterns (clusters):** North America dominates global game revenue in every cluster. Global blockbusters (Cluster 1) drive most sales. Smaller clusters show niche or regional opportunities. Japan focused games remain culturally strong but limited in volume.
- **Actionable guidance:** Prioritize NA/EU marketing and release windows for maximum ROI. Support indie and niche generes through targeted digital campaigns. Invest in cross platform franchises to reach global audiences. Localize more effetively for Japan and emerging regions to expand cluster reach.



## 9) Conclusion
- Data Driven segmentation uncovers clear market tiers in the video game industry.
- Global blockbusers dominate revenue, but niche markets offer growth potential.
